# Project 2 - Manatee Call Detection
*Ian Malone

## Project Description

The purpose of this project is to design and evaluate a machine learning detection approach to distinguish the manatee calls from the background.

The data sets as follows: 
(1) a file (train_signal.wav) with 10 different manatee calls segmented
by the biologist that represent the signal class we would like to detect
(2) a 2 second noise background file (noise_signal.wav) that represents the acoustic noise picked up by the hydrophone
(3) the continuous file (test_signal.wav) with unsegmented manatee calls mixed with background noise that lasts approximately 30 seconds. 

The sampling rate is 48 KHz

#### Steps

- Train an NLMS filter using the MCC cost function for the manatee calls train data and the background noise

## Project Implementation

#### Import Libraries

In [1]:
import scipy.io
from scipy.fft import fft, fftfreq, rfft, rfftfreq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
%load_ext autotime

time: 0 ns (started: 2021-04-10 19:54:52 -04:00)


#### Load files